# Подгрузка модели

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

#model = GPT.from_pretrained("gpt2-medium")
dismodel = AutoModelForCausalLM.from_pretrained('distilbert/distilgpt2')
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
decode = lambda l: enc.decode(l)

In [ ]:
dismodel.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
prompt = "It is just a start"

def generate(prompt,dismodel):
    model_inputs = tokenizer([prompt], return_tensors="pt").to(device)

    #dismodel.to(device)

    generated_ids = dismodel.generate(**model_inputs, max_new_tokens=200, do_sample=True)

    return tokenizer.batch_decode(generated_ids)[0].replace('\\','').replace('"<|endoftext|>','')
ans = generate(prompt)
ans

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'It is just a start" (R-USA TODAY Sports) – The Browns will be playing their home game on June 9 against the Jets, their national title will also come to Cleveland\'s home turf this offseason.The Browns need to spend as much as they can to get back to winning a Super Bowl (R-USA TODAY Sports) this offseason, according to the team\'s schedule.The Browns are set to host a wild-card game on Tuesday (June 9) at the New England Patriots.'

# Генерация 5-shot prompts

In [ ]:
import os
import pandas as pd

In [ ]:
# Install, and download MMLU
%pip install -e ../.

!curl -O https://people.eecs.berkeley.edu/~hendrycks/data.tar
!tar -xf data.tar
data_path = "data"

Obtaining file:///
ERROR: file:/// does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  158M  100  158M    0     0  66.7M      0  0:00:02  0:00:02 --:--:-- 66.7M


In [ ]:

choices = ["A", "B", "C", "D"]
sys_msg = "The following are multiple choice questions (with answers) about {}."
def create_last_prompt(question, answers):
    user_prompt = f"{question}\n" + "\n".join([f"{choice}. {answer}" for choice, answer in zip(choices, answers)]) + "\nAnswer:"
    return user_prompt

def create_example_prompt(question, answers, right_answer):
  user_prompt = f"{question}\n" + "\n".join([f"{choice}. {answer}" for choice, answer in zip(choices, answers)]) + f"\nAnswer: {right_answer}"
  return user_prompt

def n_shot_prompt(sys_msg, user_prompts, last_prompt, subject):
  return sys_msg.format(subject)+ "\n" + "\n".join([prompt for prompt in user_prompts]) + "\n" +  last_prompt


In [ ]:
questions_dir = "data/test"

In [ ]:
one_subject_questions_path = os.listdir(questions_dir)[1]
one_subject_questions_path

'high_school_macroeconomics_test.csv'

In [ ]:
def generate_few_shot_prompts(one_subject_questions_path, shot_number):
    subject = one_subject_questions_path.split('.')[0]
    df = pd.read_csv(os.path.join(questions_dir, one_subject_questions_path))
    num_prompts = len(df) // shot_number

    few_shot_prompts = []
    correct_answers = []
    for i in range(num_prompts):
        df_subset = df.iloc[i*shot_number:(i+1)*shot_number]


        user_prompts = [
            create_example_prompt(row[0], row[1:5], row[5])
            for _, row in df_subset.iterrows()
        ]


        last_prompt = create_last_prompt(df_subset.iloc[-1, 0], df_subset.iloc[-1, 1:5])
        few_shot_prompt = n_shot_prompt(sys_msg, user_prompts[:-1], last_prompt, subject)


        few_shot_prompts.append(few_shot_prompt)

        correct_answers.append(df_subset.iloc[-1, 5])

    return few_shot_prompts, correct_answers


In [ ]:
prompts, correct_answers = generate_few_shot_prompts(one_subject_questions_path, 5)
prompts[0], correct_answers[0]

('The following are multiple choice questions (with answers) about high_school_macroeconomics_test.\nWhich of the following is included in U.S. GDP? I.   The market value of movies made in Africa by U.S. citizens II.   The market value of olive oil made in Italy and sold in the United States. III.   The market value of blue jeans made in the United States and sold in Japan IV.   The market value of wine made in the United States by Canadian citizens\nA. II III and IV only\nB. I and III only\nC. II and IV only\nD. III and IV only\nAnswer: D\nWhen both short-run aggregate supply and aggregate demand increase which of the following can be said for certain?\nA. The price level rises but real GDP falls.\nB. Both the price level and real GDP rise.\nC. The price level rises but the change in real GDP is uncertain.\nD. Real GDP rises but the change in the price level is uncertain.\nAnswer: D\nTariffs and quotas\nA. result in lower domestic prices.\nB. sometimes raise and sometimes lower the am

In [ ]:
import torch

# Предполагаем, что функции encode и decode уже определены
# а также модель и устройство (device) уже инициализированы

prompt = "It is just a start"

def calculate_accuracy(test_model, prompts, correct_answers):
    global skipped
    correct_count = 0
   # model_inputs = tokenizer(prompts, return_tensors="pt").to(device)
    for prompt, correct_answer in zip(prompts, correct_answers):
        # Encode the prompt
        model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
        #x = torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...]
        try:
          generated_ids = test_model.generate(**model_inputs, max_new_tokens=1, do_sample=True)
        except:
          continue


        res = tokenizer.batch_decode(generated_ids)[0]
        model_answer = res.replace('\\','').replace('"<|endoftext|>','').strip().split()[-1].replace('n','').replace('\n','')

            # Extract the actual answer from the model's response
            #model_answer = generated_answer.strip().split()[-1]

            # Compare model's answer with the correct answer
        if model_answer == correct_answer:
            correct_count += 1
    accuracy = correct_count / len(prompts)
    return accuracy




In [ ]:
def count_acc_all_subjects(test_model, data_dir_path, num_shots):
  data_files = os.listdir(data_dir_path)
  accuracy_dict = {}
  for file in data_files:
    prompts, correct_answers = generate_few_shot_prompts(file, num_shots)
    print('prompts:',len(prompts))
    accuracy = calculate_accuracy(test_model, prompts, correct_answers)
    accuracy_dict[file] = accuracy
    print(f"Calculated for {file}, accuracy: {accuracy}")
  return accuracy_dict

In [ ]:
dismodel.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

# Подсчет Accuracy дистиляционной модели

In [ ]:
model.to(device)

In [ ]:
accuracy_dict = count_acc_all_subjects(dismodel, questions_dir, 5)
mean_accuracy = sum(accuracy_dict.values()) / len(accuracy_dict)
mean_accuracy

prompts: 54


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Token indices sequence length is longer than the specified maximum sequence length for this model (1342 > 1024). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` t

Calculated for professional_medicine_test.csv, accuracy: 0.2037037037037037
prompts: 77


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Calculated for high_school_macroeconomics_test.csv, accuracy: 0.2597402597402597
prompts: 22


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Calculated for econometrics_test.csv, accuracy: 0.3181818181818182
prompts: 108


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Calculated for high_school_psychology_test.csv, accuracy: 0.25925925925925924


0.23088270221689153

In [ ]:
import json

with open('distilation_5_shot.json', 'w') as f:
    json.dump(accuracy_dict, f)